In [2]:
import base64
import torch
from PIL import Image
from openai import OpenAI
import os
import io
from PIL import Image 
import pandas as pd

def encode_image_to_base64(image_path):
    image = Image.open(image_path)
    image = image.resize((512,512))
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")  
    image_b64=base64.b64encode(buffered.getvalue()).decode("utf-8")
    try:
        decode_img=base64.b64decode(image_b64)
    except base64.binascii.Error:
        return False
    return image_b64
def generate_instructions(image_url):
    try:
        client = OpenAI(
            api_key="",  
            base_url="https://api.openai.com/v1"
        )
        completion = client.chat.completions.create(
            model="gpt-4-turbo",
            temperature=0,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {"url": image_url}, 
                        },
                        {
                            "type": "text",
                            "text": ('''This is an image taken by a drone, showing marked waypoints for the drone's flight path. 
                            Based on the actual scene, provide flight guidance text that includes details about objects, 
                            location, edges, attributes (color, material, accessories, etc.), distance 
                            (e.g., 'long-distance flight' or 'short-distance flight'), and where to pause.
                            Format the output as Step 1, Step 2, ..., Step n. Avoid directional instructions 
                            (such as 'fly upwards') and do not mention red circles or blue lines.
                            Ensure the response is in English. Do not use directional terms or refer to points or lines not part of the scene.
                            ''')
                        },
                    ]
                }
            ],
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"An error occurred while generating instructions: {str(e)}"
image_dir="./images/"
df=pd.read_excel("./output.xlsx")
df=df.drop(index=df.index)
i=0
for item in os.listdir(image_dir):
    image_path=os.getcwd()+"/images/"+item
    print(image_path)
    image_base64=encode_image_to_base64(image_path)
    if image_base64:
        image_url=f"data:image/png;base64,{image_base64}"
        instructions = generate_instructions(image_url)
        row=pd.DataFrame({"index":i,"image_name":[item],"output_instructions":[instructions]})
    else:
        row=pd.DataFrame({"index":i,"image_name":[item],"output_instructions":[]})
    i+=1
    df = pd.concat([df,row],ignore_index=True)
    df.to_excel("./output.xlsx",index=False)


/Users/biewen/Desktop/myenv/share/images/output_path_image-08.jpeg
output_path_image-08.jpeg  output: 
 Step 1: Begin your flight near the cluster of taller buildings with red brick facades and large windows. These buildings are surrounded by streets on all sides, providing a clear area for takeoff.
Step 2: Proceed towards the area with a mix of both red and beige buildings, maintaining a steady altitude to capture the architectural diversity. These buildings are medium in height compared to the starting point.
Step 3: Continue the flight path passing over a street that intersects with several others. This area features a variety of building materials, including glass and concrete, which contrast with the earlier brick structures.
Step 4: Navigate towards a section where the buildings are predominantly lower in height. This area provides a different perspective, showcasing smaller structures amidst the urban environment.
Step 5: Pause briefly above an open space that appears to be a sm

KeyboardInterrupt: 